In [1]:
import pandas as pd
import numpy as np
import fasttext
from tqdm import tqdm

tqdm.pandas()


In [2]:
def latin_to_cyrillic(letter):
    kazakh_conversion = {
        'a': 'а',
        'á': 'ә',
        'b': 'б',
        'v': 'в',
        'g': 'г',
        'ǵ': 'ғ',
        'd': 'д',
        'e': 'е',
        'j': 'ж',
        'z': 'з',
        'ı': 'и',
        'i': 'і',
        'k': 'к',
        'q': 'қ',
        'l': 'л',
        'm': 'м',
        'n': 'н',
        'ń': 'ң',
        'o': 'о',
        'ó': 'ө',
        'p': 'п',
        'r': 'р',
        's': 'с',
        't': 'т',
        'ý': 'у',
        'u': 'ұ',
        'ú': 'ү',
        'f': 'ф',
        'h': 'х',
        'ch': 'ч',
        'sh': 'ш',
        'y': 'ы'
    }
    
    if letter[:2] in kazakh_conversion:
        return kazakh_conversion[letter[:2]]
    elif letter[0] in kazakh_conversion:
        return kazakh_conversion[letter[0]]
    else:
        return letter

def text_to_cyrillic(text):
    i = 0
    cyrillic_text = ''
    
    while i < len(text):
        # If the next two characters form a special conversion (e.g., "ch", "sh"), then process them together
        if text[i:i+2] in ['ch', 'sh']:
            cyrillic_text += latin_to_cyrillic(text[i:i+2])
            i += 2
        else:
            cyrillic_text += latin_to_cyrillic(text[i])
            i += 1
    return cyrillic_text


In [11]:
lang_detect = fasttext.load_model('lid.176.bin')

def identify_language(text):
    """Identify the language of a given text using FastText."""
    predictions = lang_detect.predict(text, k=1)  # k=1 means we want the top 1 prediction
    # The prediction will be in the format ('__label__en',), so we extract 'en' from it
    return predictions[0][0].split('__label__')[1]



## Articles

In [13]:
df_articles = pd.read_csv('case1-datasaur/epir_train/articles.csv')


In [14]:
rows_to_keep = []

for index, row in tqdm(df_articles.iterrows(), total=len(df_articles)):
    title_lang = None
    try:
        title_lang = identify_language(row['title'])
    except:
        pass

    # Condition to keep the row
    if row['sys_lang'] == 'en':
        if title_lang == 'en':
            rows_to_keep.append(index)
    else:
        rows_to_keep.append(index)
    
df_articles = df_articles.loc[rows_to_keep]


100%|████████████████████████████████| 100000/100000 [00:05<00:00, 19687.22it/s]


In [15]:
df_articles = df_articles[df_articles['title'].notna()]
df_articles = df_articles[df_articles['content'].notna()]

df_articles['title'] = df_articles['title'].str.lower()
df_articles['content'] = df_articles['content'].str.lower()

new_title = []
new_content = []
for i, row in tqdm(df_articles.iterrows(), total=len(df_articles)):
    if row['sys_lang'] == 'qq':
        new_title.append(text_to_cyrillic(row['title']))
        new_content.append(text_to_cyrillic(row['content']))
    else:
        new_title.append(row['title'])
        new_content.append(row['content'])

# df_articles['title'] = df_articles['title'].progress_apply(lambda x: text_to_cyrillic(x))
# df_articles['content'] = df_articles['content'].progress_apply(lambda x: text_to_cyrillic(x))

100%|████████████████████████████████████| 94852/94852 [02:05<00:00, 755.39it/s]


In [16]:
df_articles['title'] = new_title
df_articles['content'] = new_content

df_articles

,Unnamed: 0,id,sys_lang,projects,title,content,url
0,2884,135747,ru,departament-kkbtu-sko,бешенство – смертельная угроза для человека!,"бешенство – природно-очаговое особо опасное, с...",https://www.gov.kz/memleket/entities/departame...
1,2885,119917,ru,kgd-karaganda,порядок администрирования налога на транспортн...,в соответствии с пунктом 2 статьи 66 закона «о...,https://www.gov.kz/memleket/entities/kgd-karag...
2,2886,8414,ru,turkestan-tolebi-audany,спорт,миссия: формирование единой государственной по...,https://www.gov.kz/memleket/entities/turkestan...
3,2887,118430,kk,qazalem,қазақстанның мемлекеттік органдарымен байланыс,электрондық үкіметтің мүмкіндіктерін толық пай...,https://www.gov.kz/memleket/entities/qazalem/p...
4,2888,5319,kk,kyzylorda-kzo,2022 жылға жергілікті атқарушы органдармен сыб...,қызылорда қаласының әкімдігі р/с іс-шаралар 20...,https://www.gov.kz/memleket/entities/kyzylorda...
...,...,...,...,...,...,...,...
99995,102879,23291,ru,sko-gm,разъяснение антикоррупционного законодательства,во исполнение плана мероприятий по снижению ур...,https://www.gov.kz/memleket/entities/sko-gm/pr...
99996,102880,55210,ru,kgd-sko,статистические данные по оказанным государстве...,в течение 5 месяцев 2021 года органами государ...,https://www.gov.kz/memleket/entities/kgd-sko/p...
99997,102881,111387,kk,adilet-abai,адвокаттық қызмет,адвокаттық қызмет – адвокаттар жеке және заңды...,https://www.gov.kz/memleket/entities/adilet-ab...
99998,102882,124183,ru,abay-tabigat,заявление о намечаемой деятельности по объекту...,замечания и предложения принимаются в срок до ...,https://www.gov.kz/memleket/entities/abay-tabi...


## Life

In [19]:
df_life = pd.read_csv('case1-datasaur/epir_train/life_situations.csv')


In [20]:
rows_to_keep = []

for index, row in tqdm(df_life.iterrows(), total=len(df_life)):
    intro_lang = None
    try:
        intro_lang = identify_language(row['intro'])
    except:
        pass

    # Condition to keep the row
    if row['sys_lang'] == 'en':
        if intro_lang == 'en':
            rows_to_keep.append(index)
    else:
        rows_to_keep.append(index)
        
df_life = df_life.loc[rows_to_keep]


100%|████████████████████████████████████| 4408/4408 [00:00<00:00, 12883.42it/s]


In [21]:
df_life = df_life[df_life['intro'].notna()]
df_life = df_life[df_life['title_main'].notna()]
df_life = df_life[df_life['title_main'].notna()]
df_life = df_life[df_life['instruction'].notna()]

df_life['intro'] = df_life['intro'].str.lower()
df_life['title_main'] = df_life['title_main'].str.lower()
df_life['title_main'] = df_life['title_main'].str.lower()
df_life['instruction'] = df_life['instruction'].str.lower()


In [22]:
df_life['text1'] = df_life['title_main'] + " " + df_life['title_sub']
df_life['text2'] = df_life['intro'] + " " + df_life['instruction']
df_life.drop(columns=['intro', 'title_main', 'title_sub', 'instruction', 'subid'], inplace=True)
df_life.rename(columns={'URL': 'url'}, inplace=True)
df_life


,Unnamed: 0,id,sys_lang,url,text1,text2
20,202904,62,ru,https://beta2.egov.kz/situations/62/245?lang=ru,как установить инвалидность Прохождение медико...,"инвалиды – это лица, которые из-за стойких рас..."
21,202905,62,ru,https://beta2.egov.kz/situations/62/246?lang=ru,как установить инвалидность Необходимые докуме...,"инвалиды – это лица, которые из-за стойких рас..."
22,202906,62,ru,https://beta2.egov.kz/situations/62/248?lang=ru,как установить инвалидность Критерии установле...,"инвалиды – это лица, которые из-за стойких рас..."
23,202907,62,ru,https://beta2.egov.kz/situations/62/247?lang=ru,как установить инвалидность Установление инвал...,"инвалиды – это лица, которые из-за стойких рас..."
24,202908,62,ru,https://beta2.egov.kz/situations/62/249?lang=ru,как установить инвалидность Сроки инвалидности,"инвалиды – это лица, которые из-за стойких рас..."
...,...,...,...,...,...,...
4403,207287,225,kk,https://beta2.egov.kz/situations/225/618?lang=kk,қр президенті жанындағы мемлекеттік басқару ак...,қазақстан республикасы президенті жанындағы ме...
4404,207288,225,kk,https://beta2.egov.kz/situations/225/619?lang=kk,қр президенті жанындағы мемлекеттік басқару ак...,қазақстан республикасы президенті жанындағы ме...
4405,207289,225,kk,https://beta2.egov.kz/situations/225/621?lang=kk,қр президенті жанындағы мемлекеттік басқару ак...,қазақстан республикасы президенті жанындағы ме...
4406,207290,225,kk,https://beta2.egov.kz/situations/225/623?lang=kk,қр президенті жанындағы мемлекеттік басқару ак...,қазақстан республикасы президенті жанындағы ме...


# News

In [44]:
df_news = pd.read_csv('case1-datasaur/epir_train/news.csv')
df_news


,Unnamed: 0,id,sys_lang,projects,title,body,short_description,url
0,102884,117143,qq,mangystau,MAŃǴYSTAÝDA I ZhÁNE II DEŃGEILI INFEKTsIIaLYQ ...,Mańǵystaýda 4 qarashadaǵy zhaǵdaiǵa sáikes I-I...,NaN,https://www.gov.kz/memleket/entities/mangystau...
1,102885,432491,ru,vko-glubokoe,Что необходимо знать избирателю?,20 ноября 2022 года состоится важное событие в...,NaN,https://www.gov.kz/memleket/entities/vko-glubo...
2,102886,563257,qq,turkestan-shardarin-audany,FÝTBOL ZhARYSY UIYMDASTYRYLDY.,Zhastar arasynda salaýatty ómir saltyn qalypta...,NaN,https://www.gov.kz/memleket/entities/turkestan...
3,102887,591387,ru,mvd-almaty,ДП Алматы: ежедневно общественный порядок в го...,"Как ранее сообщалось, преступность в Алматы сн...",NaN,https://www.gov.kz/memleket/entities/mvd-almat...
4,102888,312155,kk,almaty,Б. Сағынтаев ҰҚШҰ Ұжымдық бітімгершілік күштер...,Алматы қаласында Ұжымдық қауіпсіздік туралы ша...,NaN,https://www.gov.kz/memleket/entities/almaty/pr...
...,...,...,...,...,...,...,...,...
99995,202879,367308,kk,borodulihamaslikhat,"Аудандық мәслихаттың хатшысы, «Аманат» партияс...","Аудандық мәслихаттың хатшысы, «Аманат» партияс...",NaN,https://www.gov.kz/memleket/entities/borodulih...
99996,202880,589811,qq,maslihat-almaty,Almaty qalasy Máslihaty depýtattarynyń qoǵamdy...,Qurmetti almatylyqtar!⠀🔹 Nazarlaryńyzǵa #Masli...,NaN,https://www.gov.kz/memleket/entities/maslihat-...
99997,202881,259111,en,mfa-bratislava,Strengthening scientific research cooperation ...,"ZVOLEN, September 21, 2021 – As part of streng...",NaN,https://www.gov.kz/memleket/entities/mfa-brati...
99998,202882,592904,ru,abay-bilim,Сообщать о коррупции очень важно,Антикоррупционной стратегией РК на 2015-2025 г...,NaN,https://www.gov.kz/memleket/entities/abay-bili...


In [45]:
rows_to_keep = []

for index, row in tqdm(df_news.iterrows(), total=len(df_news)):
    title_lang = content_lang = None
    try:
        title_lang = identify_language(row['title'])
    except:
        pass

    # Condition to keep the row
    if row['sys_lang'] == 'en':
        if title_lang == 'en':
            rows_to_keep.append(index)
    else:
        rows_to_keep.append(index)        


100%|████████████████████████████████| 100000/100000 [00:05<00:00, 19176.69it/s]


In [46]:
df_news = df_news.loc[rows_to_keep]

In [47]:
df_news = df_news[df_news['title'].notna()]
df_news = df_news[df_news['body'].notna()]

df_news['title'] = df_news['title'].str.lower()
df_news['body'] = df_news['body'].str.lower()

new_title = []
new_content = []
for i, row in tqdm(df_news.iterrows(), total=len(df_news)):
    if row['sys_lang'] == 'qq':
        new_title.append(text_to_cyrillic(row['title']))
        new_content.append(text_to_cyrillic(row['body']))
    else:
        new_title.append(row['title'])
        new_content.append(row['body'])


100%|███████████████████████████████████| 95021/95021 [00:57<00:00, 1664.46it/s]


In [48]:
df_news['title'] = new_title
df_news['body'] = new_content


In [51]:
df_news.drop(columns=['short_description', 'projects'], inplace=True)


In [52]:
df_news

,Unnamed: 0,id,sys_lang,title,body,url
0,102884,117143,qq,маңғыстауда і зхәне іі деңгеілі інфектсііалық ...,маңғыстауда 4 қарашадағы зхағдаіға сәікес і-іі...,https://www.gov.kz/memleket/entities/mangystau...
1,102885,432491,ru,что необходимо знать избирателю?,20 ноября 2022 года состоится важное событие в...,https://www.gov.kz/memleket/entities/vko-glubo...
2,102886,563257,qq,футбол зхарысы ұіымдастырылды.,зхастар арасында салауатты өмір салтын қалыпта...,https://www.gov.kz/memleket/entities/turkestan...
3,102887,591387,ru,дп алматы: ежедневно общественный порядок в го...,"как ранее сообщалось, преступность в алматы сн...",https://www.gov.kz/memleket/entities/mvd-almat...
4,102888,312155,kk,б. сағынтаев ұқшұ ұжымдық бітімгершілік күштер...,алматы қаласында ұжымдық қауіпсіздік туралы ша...,https://www.gov.kz/memleket/entities/almaty/pr...
...,...,...,...,...,...,...
99995,202879,367308,kk,"аудандық мәслихаттың хатшысы, «аманат» партияс...","аудандық мәслихаттың хатшысы, «аманат» партияс...",https://www.gov.kz/memleket/entities/borodulih...
99996,202880,589811,qq,алматы қаласы мәсліхаты депутаттарының қоғамды...,құрметті алматылықтар!⠀🔹 назарларыңызға #маслі...,https://www.gov.kz/memleket/entities/maslihat-...
99997,202881,259111,en,strengthening scientific research cooperation ...,"zvolen, september 21, 2021 – as part of streng...",https://www.gov.kz/memleket/entities/mfa-brati...
99998,202882,592904,ru,сообщать о коррупции очень важно,антикоррупционной стратегией рк на 2015-2025 г...,https://www.gov.kz/memleket/entities/abay-bili...


In [54]:
df_news = df_news[df_news['sys_lang'].isin(['en', 'kk', 'qq', 'ru'])]

In [58]:
df_articles.rename(columns={'title': 'text1', 'content': 'text2'}, inplace=True)

In [61]:
df_services = pd.read_csv('case1-datasaur/epir_train/services.csv')
df_services


,Unnamed: 0,id,sys_lang,additional_info,description,full_title,short_title,title,result_description,url
0,0,3087,en,Rules for providing state services,Dear citizens of the Republic of Kazakhstan! I...,"Issuance of passports, Identification cards fo...","Obtainment of passport, national ID","Obtainment of passport, national ID",Issuance of passport and (or) national ID of a...,https://beta2.egov.kz/services/3087?lang=en
1,1,3081,en,Rules for providing state services,Information is being updated. Citizens of the ...,Acceptance of documents for issuance of passpo...,Acceptance of documents for issuance of passpo...,Acceptance of documents for issuance of passpo...,Passport of a citizen of the Republic of Kazak...,https://beta2.egov.kz/services/3081?lang=en
2,2,4632,en,NaN,NaN,P250.00,P250.00 тест 996,P250.00,NaN,https://beta2.egov.kz/services/4632?lang=en
3,3,4617,en,NaN,NaN,CR.01,CR.01,CR.01,NaN,https://beta2.egov.kz/services/4617?lang=en
4,4,3811,en,NaN,Information is being updated. What is Personal...,Request for Personal Record,Request for Personal Record,Request for Personal Record,"Personal Record from ""e-Kyzmet"" IIS.",https://beta2.egov.kz/services/3811?lang=en
...,...,...,...,...,...,...,...,...,...,...
2879,2879,4732,ru,NaN,NaN,P7.05,P7.05,P7.05,NaN,https://beta2.egov.kz/services/4732?lang=ru
2880,2880,4733,ru,NaN,NaN,P3.03,P3.03,P3.03,NaN,https://beta2.egov.kz/services/4733?lang=ru
2881,2881,4735,ru,NaN,NaN,P3.041,P3.041,P3.041,NaN,https://beta2.egov.kz/services/4735?lang=ru
2882,2882,4736,ru,NaN,NaN,P21.09,P21.09,P21.09,NaN,https://beta2.egov.kz/services/4736?lang=ru


In [73]:
df_services = df_services[df_services['description'].notna()]
df_services = df_services[df_services['result_description'].notna()]
df_services = df_services[df_services['full_title'].notna()]
df_services = df_services[df_services['short_title'].notna()]
df_services = df_services[df_services['title'].notna()]
df_services = df_services[df_services['additional_info'].notna()]


In [74]:
df_services['description'] = df_services['description'].str.lower()
df_services['result_description'] = df_services['result_description'].str.lower()
df_services['full_title'] = df_services['full_title'].str.lower()
df_services['short_title'] = df_services['short_title'].str.lower()
df_services['title'] = df_services['title'].str.lower()
df_services['additional_info'] = df_services['additional_info'].str.lower()


In [75]:
rows_to_keep = []

for index, row in tqdm(df_services.iterrows(), total=len(df_services)):
    title_lang = None
    try:
        title_lang = identify_language(row['full_title'])
    except:
        pass

    # Condition to keep the row
    if row['sys_lang'] == 'en':
        if title_lang == 'en':
            rows_to_keep.append(index)
    else:
        rows_to_keep.append(index)
        

100%|████████████████████████████████████| 2326/2326 [00:00<00:00, 18207.35it/s]


In [76]:
df_services = df_services.loc[rows_to_keep]

In [79]:
df_services['text1'] = df_services['additional_info'] + " " + df_services['description'] + " " + df_services['result_description']
df_services['text2'] = df_services['full_title'] + " " + df_services['short_title'] + " " + df_services['title']

df_services.drop(columns=['additional_info', 'description', 'full_title', 'short_title', 'title', 'result_description'], inplace=True)


In [80]:
df_services

,Unnamed: 0,id,sys_lang,url,text1,text2
0,0,3087,en,https://beta2.egov.kz/services/3087?lang=en,rules for providing state services dear citize...,"issuance of passports, identification cards fo..."
1,1,3081,en,https://beta2.egov.kz/services/3081?lang=en,rules for providing state services information...,acceptance of documents for issuance of passpo...
5,5,3263,en,https://beta2.egov.kz/services/3263?lang=en,rules for providing state services information...,obtaining a certificate on the status of the i...
6,6,3040,en,https://beta2.egov.kz/services/3040?lang=en,rules for providing state services dear citize...,removal from registration at the place of resi...
7,7,3759,en,https://beta2.egov.kz/services/3759?lang=en,rules for providing state service what is temp...,issuance of a list on temporary work incapacit...
...,...,...,...,...,...,...
2799,2799,3129,kk,https://beta2.egov.kz/services/3129?lang=kk,қызмет стандарты ақпарат өзектендіру сатысында...,қазақстан республикасында уақытша болатын шете...
2802,2802,3140,qq,https://beta2.egov.kz/services/3140?lang=qq,memlekettik qyzmet kórsetý erezhesi aqparat óz...,qazaqstan respýblikasynda bosqyn mártebesin be...
2803,2803,3038,qq,https://beta2.egov.kz/services/3038?lang=qq,memlekettik qyzmet kórsetý erezhesi qurmetti q...,qazaqstan respýblikasynyń halqyn turǵylyqty zh...
2867,2867,3570,qq,https://beta2.egov.kz/services/3570?lang=qq,memlekettik qyzmet kórsetý erezhesi qurmetti q...,investitsiialardy zhúzege asyrýdy zháne invest...


In [83]:
df_news.rename(columns={'title': 'text1', 'body': 'text2'}, inplace=True)

/tmp/ipykernel_28133/3265959969.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_news.rename(columns={'title': 'text1', 'body': 'text2'}, inplace=True)


In [88]:
df = pd.concat([df_services, df_articles, df_news, df_life])

In [91]:
df.drop(['projects'], axis=1, inplace=True)

In [93]:
df

,Unnamed: 0,id,sys_lang,url,text1,text2
0,0,3087,en,https://beta2.egov.kz/services/3087?lang=en,rules for providing state services dear citize...,"issuance of passports, identification cards fo..."
1,1,3081,en,https://beta2.egov.kz/services/3081?lang=en,rules for providing state services information...,acceptance of documents for issuance of passpo...
5,5,3263,en,https://beta2.egov.kz/services/3263?lang=en,rules for providing state services information...,obtaining a certificate on the status of the i...
6,6,3040,en,https://beta2.egov.kz/services/3040?lang=en,rules for providing state services dear citize...,removal from registration at the place of resi...
7,7,3759,en,https://beta2.egov.kz/services/3759?lang=en,rules for providing state service what is temp...,issuance of a list on temporary work incapacit...
...,...,...,...,...,...,...
4403,207287,225,kk,https://beta2.egov.kz/situations/225/618?lang=kk,қр президенті жанындағы мемлекеттік басқару ак...,қазақстан республикасы президенті жанындағы ме...
4404,207288,225,kk,https://beta2.egov.kz/situations/225/619?lang=kk,қр президенті жанындағы мемлекеттік басқару ак...,қазақстан республикасы президенті жанындағы ме...
4405,207289,225,kk,https://beta2.egov.kz/situations/225/621?lang=kk,қр президенті жанындағы мемлекеттік басқару ак...,қазақстан республикасы президенті жанындағы ме...
4406,207290,225,kk,https://beta2.egov.kz/situations/225/623?lang=kk,қр президенті жанындағы мемлекеттік басқару ак...,қазақстан республикасы президенті жанындағы ме...


In [97]:
df.to_csv('df.csv', index=0)

In [98]:
pd.read_csv('df.csv')

,Unnamed: 0,id,sys_lang,url,text1,text2
0,0,3087,en,https://beta2.egov.kz/services/3087?lang=en,rules for providing state services dear citize...,"issuance of passports, identification cards fo..."
1,1,3081,en,https://beta2.egov.kz/services/3081?lang=en,rules for providing state services information...,acceptance of documents for issuance of passpo...
2,5,3263,en,https://beta2.egov.kz/services/3263?lang=en,rules for providing state services information...,obtaining a certificate on the status of the i...
3,6,3040,en,https://beta2.egov.kz/services/3040?lang=en,rules for providing state services dear citize...,removal from registration at the place of resi...
4,7,3759,en,https://beta2.egov.kz/services/3759?lang=en,rules for providing state service what is temp...,issuance of a list on temporary work incapacit...
...,...,...,...,...,...,...
196129,207287,225,kk,https://beta2.egov.kz/situations/225/618?lang=kk,қр президенті жанындағы мемлекеттік басқару ак...,қазақстан республикасы президенті жанындағы ме...
196130,207288,225,kk,https://beta2.egov.kz/situations/225/619?lang=kk,қр президенті жанындағы мемлекеттік басқару ак...,қазақстан республикасы президенті жанындағы ме...
196131,207289,225,kk,https://beta2.egov.kz/situations/225/621?lang=kk,қр президенті жанындағы мемлекеттік басқару ак...,қазақстан республикасы президенті жанындағы ме...
196132,207290,225,kk,https://beta2.egov.kz/situations/225/623?lang=kk,қр президенті жанындағы мемлекеттік басқару ак...,қазақстан республикасы президенті жанындағы ме...
